In [25]:
import requests, json, numpy, datetime
import pywren
import boto3
import numpy as np
from pyproj import Proj
import warnings
warnings.filterwarnings('ignore')

In [32]:
BUCKET = 'mapping-disaster-risk'

GEOJSONS = ['train-borde_rural.geojson', 'train-borde_soacha.geojson',
            'train-castries.geojson', 'train-dennery.geojson',
            'train-gros_islet.geojson', 'train-mixco_1_and_ebenezer.geojson',
            'train-mixco_3.geojson']

TIFS = ['borde_rural_ortho-cog.tif', 'borde_soacha_ortho-cog.tif',
        'castries_ortho-cog.tif', 'dennery_ortho-cog.tif',
        'gros_islet_ortho-cog.tif', 'mixco_1_and_ebenezer_ortho-cog.tif',
        'mixco_3_ortho-cog.tif']

tif_list = [{'tif':"borde_rural_ortho-cog.tif",
    'crs': 'epsg:32618',
    'geojson': 'train-borde_rural.geojson',
    'output': 'colombia_rural_zonal.csv'},
    {'tif':"borde_soacha_ortho-cog.tif",
    'crs': 'epsg:32618',
    'geojson':'train-borde_soacha.geojson',
    'output': 'colombia_soacha_zonal.csv'},
    {'tif':"mixco_1_and_ebenezer_ortho-cog.tif",
    'crs': 'epsg:32616',
    'geojson': 'train-mixco_1_and_ebenezer.geojson',
    'output': 'guatemala_mixco_1_zonal.csv'},
    {'tif':"mixco_3_ortho-cog.tif",
    'crs': 'epsg:32616',
    'geojson': 'train-mixco_3.geojson',
    'output': 'guatemala_mixco_3_zonal.csv'},
    {'tif':"castries_ortho-cog.tif",
    'crs': 'epsg:32620',
    'geojson': 'train-castries.geojson',
    'output': 'stlucia_castries_zonal.csv'},
    {'tif':"dennery_ortho-cog.tif",
    'crs': 'epsg:32620',
    'geojson': 'train-dennery.geojson',
    'output': 'stlucia_dennery_zonal.csv'},
    {'tif':"gros_islet_ortho-cog.tif",
    'crs': 'epsg:32620',
    'geojson': 'train-gros_islet.geojson',
    'output': 'stlucia_gros_islet_zonal.csv'}]

In [ ]:
# Note that all geojsonn names are the object names as well, so to access an object call:
# https://mapping-disaster-risk.s3.amazonaws.com/train-borde_rural.geojson

In [30]:
# Step 1: Get all geojsons and create new footprint dictionaries
def get_geojsons(dictionary):
    '''
    Given a full geojson dictionary, extracts relevant information for each
    building, including type, building id, roof material, and coordinates.
    Returns a list of dictionaries, where each dictionary maps a building by
    these features.
   
    Input:
        geojson (str): name of geojson dictionary
    
    Output:
        polygons (lst): a list of dictionaries (one per building)
    '''
    geojson = dictionary['geojson']
    tif = dictionary['tif']
    crs = dictionary['crs']
    output = dictionary['output']

    try:
        s3 = boto3.client('s3', 'us-east-1')
        obj = s3.get_object(Bucket='mapping-disaster-risk', Key=geojson)
        geo_json = json.load(obj['Body'])
        polygons = []
        for feature in geo_json['features']:
            polygon = {}
            polygon['type'] = feature['geometry']
            polygon['bid'] = feature['id']
            polygon['roof_material'] = feature['properties']['roof_material']
            polygon['coordinates'] = feature['geometry']['coordinates']
            polygon['geojson'] = geojson
            polygon['tif'] = tif
            polygon['crs'] = crs
            polygon['output'] = output
            polygons.append(polygon)
    except Exception as e:
        return e

    return polygons

In [20]:
def transform_coordinates(polygon_list):
    '''
    Given building footprints, executes an affine transformation on each
    coordinate pair to output a new coordinate pair based on a provided
    coordinate reference system (crs) from the corresponding geotiff file.
    
    Input:
        polygon_list (lst): one dictionary footprint per building

    Output:
        updated_polygons (lst): a list of dictionaries (one per building)
    '''

    geojson = polygon_list[0]['geojson']
    crs = polygon_list[0]['crs']
    tif = polygon_list[0]['tif']
    transform = Proj(init=crs)

    updated_polygons = []
    for polygon in polygon_list:
        for coordinates_list in polygon['coordinates']:
            inner_coord_list = []
            for coordinate in coordinates_list:
                transformed_coordinate = list(transform(coordinate[0],
                                                        coordinate[1]))
                inner_coord_list.append(transformed_coordinate)        
            polygon['coordinates'] = inner_coord_list
            updated_polygon_list.append(polygon)

    return updated_polygons

In [ ]:
def get_tif():
    pass

In [ ]:
def get_rooftop_array_after_mask():
    pass

In [ ]:
def convert_grayscale():
    pass

In [ ]:
def crop_center():
    pass

In [ ]:
def calculate_zonal_stats():
    pass

In [ ]:
def build_df():
    pass

In [33]:
# Obtain footprint dictionaries
# Returns a list of lists, where each list are polygons for one .tif file
pwex = pywren.default_executor()
futures = pwex.map(get_geojsons, tif_list)
polygons = pywren.get_all_results(futures)

In [22]:
# Transform coordinates
polygon_futures = pwex.map(transform_coordinates, polygons)
updated_polygons = pywren.get_all_results(polygon_futures)

ModuleNotFoundError: No module named 'pyproj'

In [16]:
updated_polygons[0]